# Normalização dos dados dos pacientes
---
Este arquivo realiza o tratamento dos dados coletados em diferentes planilhas e executa as seguintes tarefas:
- Geração de um único arquivo CSV contendo as características clínicas e laboratoriais;
- Filtra os dados para obter somente os primeiros 20 dias de internação de cada paciente;
- Normalização dos valores;
- Remoção de coletas inválidas;
- **OneHotEncoding** das características multivaloradas

#### 1 - Importações
---
Importaçõo das biliotecas utilizas no procedimento

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
from datetime import date
import math

#### 2 - Rotinas auxiliares
---
Declaração das rotinas para tratamento de datas e números no conjunto de dados

In [2]:
def data_valida(data):
    if ((data == '') or pd.isna(data)):
        return False    
    
    dataValida = None
    try:
        dt.datetime.strptime(data, "%d/%m/%y")
        dataValida = True
    except ValueError:
        try:
            dt.datetime.strptime(data, "%d/%m/%Y")
            dataValida = True
        except ValueError:
            dataValida = False
    return dataValida

def obter_data(data):
    try:
        if(isinstance(data, dt.datetime)):
            return data;
            
        return dt.datetime.strptime(data, "%d/%m/%y")      
    except ValueError:
        try:
            return dt.datetime.strptime(data, "%d/%m/%Y")
        except ValueError:
            #Retorna uma data que irá falhar a validação
            return dt.datetime.strptime("01/01/1990", "%d/%m/%Y")

def numero_valido(s):
    try:
        if (math.isnan(float(s))):
            return False
        float(s)
        return True
    except:
        return False
    
def diferenca_entre(data1, data2):
    return abs(obter_data(data2) - obter_data(data1)).days

#### 3 - Leitura dos arquivos brutos
---
Inclusão das variáveis `prontuarios_bruto` e `coletas_bruto` com os dados originais dos arquivos CSV coletados nos hospitais

In [3]:
prontuarios_bruto = pd.read_csv('./datasets/bruto_prontuarios.csv', delimiter=',', index_col=False)
coletas_bruto = pd.read_csv('./datasets/bruto_laboratoriais.csv', delimiter=',')

#### 4 - Tratamento dos dados
---
Nesta etapa os dados são validados e cada coleta de dados laboratoriais é validada e unida aos dados do prontuário do paciente. Coletas após os primeiros 20 dias de internação ou que não possuem valores válidos são descartadas. Ao final, um `DataFrame` de nome `dataset` é criado com os dados filtrados.

In [4]:
id = np.nan
registros = []
data_inicial = dt.datetime.strptime("01/01/2020", "%d/%m/%Y")

for i in range(len(coletas_bruto)):
    linha_coleta = coletas_bruto.loc[i, :]
        
    #Se é o primeiro registro ou mudou o paciente, procura os dados do prontuário
    if ((pd.isna(id) or id != linha_coleta[0]) and (pd.isna(linha_coleta[0]) == False)):
        id = int(linha_coleta[0])
        prontuario = prontuarios_bruto.loc[prontuarios_bruto['ID'] == id]
               
        #Se não encontrou um prontuário pro paciente, ignora a coleta
        if (prontuario.empty == True):
            continue;            
        
        linha_prontuario = prontuario.iloc[0, :]
        
        #Desconsidera prontuários sem uma das colunas preenchidas
        if (pd.isna(linha_prontuario['Tempo Internação']) or pd.isna(linha_prontuario['IDADE']) or 
            pd.isna(linha_prontuario['SEXO']) or pd.isna(linha_prontuario['COMORBIDADES']) or
            pd.isna(linha_prontuario['DATA'])):
            continue;            
            
        #Desconsidera linhas onde a data de entrada não é válida
        if (data_valida(linha_prontuario['DATA']) == False):
            continue;            
            
        data_inicial = linha_prontuario['DATA'];        
    
    #Verifica se ainda está no mesmo paciente
    if (id == linha_coleta[0] or pd.isna(linha_coleta[0])):       
        #Desconsidera linhas onde não há coleta
        if (data_valida(linha_coleta[1]) == False):
            continue;
                    
        #Desconsidera exames com mais de 20 dias da data inicial do paciente
        if (diferenca_entre(data_inicial, linha_coleta[1]) > 20):
            #print(str(id) + ': ' + data_inicial + ' - ' + linha_coleta[1] + ' = ' + str(diferenca_entre(data_inicial, linha_coleta[1])))        
            continue;
            
        #Desconsidera prontuários sem uma das colunas preenchidas
        if (pd.isna(linha_prontuario['Tempo Internação']) or pd.isna(linha_prontuario['IDADE']) or 
            pd.isna(linha_prontuario['SEXO']) or pd.isna(linha_prontuario['COMORBIDADES']) or
            pd.isna(linha_prontuario['DATA'])):
            continue;   
            
        coletas_dados = coletas_bruto.drop('Número Paciente', inplace=False, axis=1).drop('Data', inplace=False, axis=1).loc[i, :]
                
        registros.append(
            {
                'tempo_internacao': int(linha_prontuario['Tempo Internação']),
                'idade': int(linha_prontuario['IDADE']),
                'sexo': int(linha_prontuario['SEXO']) - 1,
                'dm': 1 if (int(linha_prontuario['COMORBIDADES']) in [1, 6, 7, 8]) else 0,
                'has': 1 if (int(linha_prontuario['COMORBIDADES']) in [2, 5, 6, 8]) else 0,
                'obesidade': 1 if (int(linha_prontuario['COMORBIDADES']) in [3, 5, 7, 8]) else 0,
                'neoplasia': 1 if (int(linha_prontuario['COMORBIDADES']) == 4) else 0
            })
                
        for k in range(len(coletas_dados)):
            try:                
                valor = 0 if (not numero_valido(str(coletas_dados[k]).replace(',', '.'))) else float(str(coletas_dados[k]).replace(',', '.'))
            except:
                valor = 0
                
            registros[-1][coletas_dados.axes[0][k]] = valor
        
        #adiciona o desfecho como última coluna
        registros[-1]['obito'] = 1 if linha_prontuario['DESFECHO'] == 2 else 0
        
dataset = pd.DataFrame(registros)

#### 5 - Apresentação do conjunto final
---
O conjunto de dados finalizado é apresentado para validação

In [5]:
dataset.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1102 entries, 0 to 1101
Data columns (total 107 columns):
 #    Column                 Dtype  
---   ------                 -----  
 0    tempo_internacao       int64  
 1    idade                  int64  
 2    sexo                   int64  
 3    dm                     int64  
 4    has                    int64  
 5    obesidade              int64  
 6    neoplasia              int64  
 7    Eritrócitos            float64
 8    Hemoglobina            float64
 9    Hematócrito            float64
 10   VCM                    float64
 11   HCM                    float64
 12   CHCM                   float64
 13   RDW                    float64
 14   Leucócitos             float64
 15   Blastos                float64
 16   Promielócitos          float64
 17   Mielócitos             float64
 18   Metamielócitos         float64
 19   Bastóes                float64
 20   Segmentados            float64
 21   Eosinófilos            float64
 22 

#### 6 - Salvamento em disco
---
O dataset final é salvo em disco para processamento na próxima etapa

In [6]:
dataset.to_csv('./datasets/dataset_primerios_20_dias.csv', index=False)